In [5]:
import os
import time
import warnings
import numpy as np
import random as rnd
import pandas as pd
from collections import defaultdict

# Librería Genética
from deap import base, creator, tools, algorithms

from sklearn.utils import shuffle
# Subfunciones de estimadores
from sklearn.base import clone
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/base.py][30]
from sklearn.base import is_classifier
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/base.py][535]
from sklearn.model_selection._validation import _fit_and_score
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_validation.py][346]
from sklearn.model_selection._search import BaseSearchCV
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py][386]
from sklearn.model_selection._search import check_cv
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_split.py][1866]
from sklearn.model_selection._search import _check_param_grid
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py][343]
from sklearn.metrics.scorer import check_scoring
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/metrics/scorer.py][250]
from sklearn.utils.validation import _num_samples
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/validation.py][105]
from sklearn.utils.validation import indexable
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/validation.py][208]
from multiprocessing import Pool, Manager, cpu_count

# Selección para estimadores
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Metricas para estimadores
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

# Estimadores
from sklearn.ensemble import ExtraTreesClassifier#
from sklearn.ensemble import RandomForestClassifier#
from sklearn.ensemble import AdaBoostClassifier#
from sklearn.ensemble import GradientBoostingClassifier#
from sklearn.tree import DecisionTreeClassifier#
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

"""
PYMACH
"""
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import FunctionTransformer

#Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#Ensembles algorithms
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
warnings.filterwarnings("ignore")

# find distance error al 0.2%
def distance_error(estimator, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 7)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    # coord pred
    x1 = np.int32((y_pred + 2) % 3)
    y1 = np.int32((y_pred - 1) / 3)
    # coord real
    x2 = np.int32((y_test + 2) % 3)
    y2 = np.int32((y_test - 1) / 3)
    # pasar variacion a distancias metros
    vx = np.abs(x1 - x2)
    vy = np.abs(x1 - x2)
    vx = vx*0.5 + (vx-1)*(vx>0)
    vy = vy*0.5 + (vy-1)*(vy>0)
    # pitagoras
    err_distance = np.mean(np.sqrt(vx*vx + vy*vy))
    return err_distance

# The problem to optimize
def evaluate( frecuencias, individual, estimator, score_cache={}, error_cache={}, 
             n_splits = 10, shuffle = False, scorer = "accuracy"):
    X, y = _createDataset(frecuencias, individual)
    score = 0
    name = str(estimator).split('(')[0]
    paramkey = name+str(individual)
    if paramkey in score_cache:
        score = score_cache[paramkey]
        error = error_cache[paramkey]
    else:
        kfold = KFold(n_splits=n_splits, shuffle=shuffle)
        cv_results = cross_val_score(estimator, X, y, cv=kfold, scoring=scorer)
        score = cv_results.mean()
        error = cv_results.std()
        metric_err = distance_error(estimator, X, y)
        results = {'Modelo': name, 'Configuracion':np.int32(individual)+1, 'values': cv_results, 'Accuracy': score, 'stdAccuracy': error, 'errorMetrico': metric_err}
        #score_cache[paramkey] = score
        #error_cache[paramkey] = error
    return score, results


def _createDataset(frecuencias, values, seed = 7):
    # crear dataset
    names_ = frecuencias[0].columns.values
    # reestructuracion
    salida_final = pd.DataFrame(columns=names_)
    for sec in range(1,16):
        dataset = pd.DataFrame(columns=names_)
        corte = min([frecuencias[i][frecuencias[i]['Sector']==sec].shape[0] for i in values])
        tx = 0
        dataset[names_[tx]] = dataset[names_[tx]].append(frecuencias[int(values[tx])][frecuencias[int(values[tx])]['Sector']==sec][:corte][names_[tx]])
        for tx in range(1,5):
            dataset[names_[tx]] = frecuencias[int(values[tx])][frecuencias[int(values[tx])]['Sector']==sec][:corte][names_[tx]]
        dataset[names_[tx+1]] = frecuencias[int(values[tx])][frecuencias[int(values[tx])]['Sector']==sec][:corte][names_[tx+1]]
        # join parts
        salida_final = salida_final.append(dataset)
    # shuffle dataset
    salida_final = shuffle(salida_final, random_state=seed).reset_index(drop=True)
    salida_final = salida_final.apply(pd.to_numeric)
    # dataframe to X,y 
    X = salida_final[names_[:-1]]
    y = salida_final[names_[-1]]
    return X,y

def set_models():
    rs = 1
    models = []
    # LDA : Warning(Variables are collinear)
    models.append(('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()))
    
    models.append(('SVC', SVC(random_state=rs)))
    models.append(('GaussianNB', GaussianNB()))
    models.append(('MLPClassifier', MLPClassifier()))
    models.append(('KNeighborsClassifier', KNeighborsClassifier()))
    models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state=rs)))
    models.append(('LogisticRegression', LogisticRegression()))
    # Bagging and Boosting
    # models.append(('ExtraTreesClassifier', ExtraTreesClassifier(n_estimators=150)))
    models.append(('ExtraTreesClassifier', ExtraTreesClassifier(random_state=rs)))
    models.append(('AdaBoostClassifier', AdaBoostClassifier(DecisionTreeClassifier(random_state=rs),
                                                            random_state=rs)))
    # models.append(('AdaBoostClassifier', AdaBoostClassifier(DecisionTreeClassifier())))
    models.append(('RandomForestClassifier', RandomForestClassifier(random_state=rs)))
    models.append(('GradientBoostingClassifier',
                   GradientBoostingClassifier(random_state=rs)))
    # models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
    # Voting
    estimators = []
    estimators.append(("Voting_GradientBoostingClassifier", GradientBoostingClassifier(random_state=rs)))
    estimators.append(("Voting_ExtraTreesClassifier", ExtraTreesClassifier(random_state=rs)))
    voting = VotingClassifier(estimators)
    models.append(('VotingClassifier', voting))
    """
    return models

In [2]:
test_size = 0.2
num_folds = 10
seed = 7
frecuencias = []
names_ = ['Be01', 'Be02', 'Be03', 'Be04', 'Be05', 'Sector']
frecuencias.append(pd.read_csv('sinFiltro/Tx_0x01'))#, names=names_))
frecuencias.append(pd.read_csv('sinFiltro/Tx_0x02'))#, names=names_))
frecuencias.append(pd.read_csv('sinFiltro/Tx_0x03'))#, names=names_))
frecuencias.append(pd.read_csv('sinFiltro/Tx_0x04'))#, names=names_))
frecuencias.append(pd.read_csv('sinFiltro/Tx_0x05'))#, names=names_))
frecuencias.append(pd.read_csv('sinFiltro/Tx_0x06'))#, names=names_))
num_jobs = cpu_count()
estimadores = set_models()

In [6]:
"""
    def build_pipelines(self):
        pipelines = []
        models = self.set_models()

        for m in models:
            pipelines.append((m[0],
                Pipeline([
                    ('preparer', self.preparer),
                    m,
                ])
            ))

        self.pipelines = pipelines

        return pipelines
"""
salida = []
for name, model in estimadores:
    print("\nModeling...", name)
    splits = 10
    simetricas = [[i]*5 for i in range(6)]
    for individual in simetricas:
        acc, results = evaluate(frecuencias, individual, model)
        #salida[str(name)+"-"+str(individual)] = str(acc) + "-"+ str(desv) + "-" + str(err)
        salida.append(results)
        print(results['Modelo']," ", individual, "\t", results['Accuracy'], "\t", results['stdAccuracy'], "\t", results['errorMetrico'])



Modeling... LinearDiscriminantAnalysis
LinearDiscriminantAnalysis   [0, 0, 0, 0, 0] 	 0.638416766467 	 0.0187798305257 	 0.32423777479
LinearDiscriminantAnalysis   [1, 1, 1, 1, 1] 	 0.519638313341 	 0.0214421123982 	 0.443793684573
LinearDiscriminantAnalysis   [2, 2, 2, 2, 2] 	 0.578655533782 	 0.0199728943849 	 0.359756081656
LinearDiscriminantAnalysis   [3, 3, 3, 3, 3] 	 0.625618358477 	 0.0210842743655 	 0.358028749968
LinearDiscriminantAnalysis   [4, 4, 4, 4, 4] 	 0.543107562491 	 0.0243615675336 	 0.50400164191
LinearDiscriminantAnalysis   [5, 5, 5, 5, 5] 	 0.666508125923 	 0.0209476336107 	 0.426789450073

Modeling... SVC
SVC   [0, 0, 0, 0, 0] 	 0.722175648703 	 0.0168578958543 	 0.305871364889
SVC   [1, 1, 1, 1, 1] 	 0.687189749182 	 0.0209334522397 	 0.253211571168
SVC   [2, 2, 2, 2, 2] 	 0.748960126097 	 0.023039128306 	 0.240810358918
SVC   [3, 3, 3, 3, 3] 	 0.740944395143 	 0.0111034605247 	 0.289177067282
SVC   [4, 4, 4, 4, 4] 	 0.671918069228 	 0.0250041277738 	 0.2911173

In [18]:
df = pd.DataFrame.from_dict(salida, orient='index')
df.reset_index(level=0, inplace=True)
#display(df.sort_values([0],ascending=False).head(30))
# Guarda el dataframe en un archivo csv
#df.sort_values([0],ascending=False).to_csv('eas.csv', sep=',', index=False) 
#df = pd.DataFrame(df.index.str.split('-',1).tolist(), columns = ['index'])
s = df['index'].apply(lambda x: x.split('-'))
df['Modelo'] = s.apply(lambda x: x[0])
df['Configuracion'] = s.apply(lambda x: x[1])
t = df[0].apply(lambda x: x.split('-'))
df['Precision'] = t.apply(lambda x: x[0])
df['desvPrecision'] = t.apply(lambda x: x[1])
df['errorMetrico'] = t.apply(lambda x: x[2])
df = df.drop(['index', 0], axis=1)
df = df.sort_values(['Precision'],ascending=False)
df.to_csv('resultados_simetricos_sinFiltro.csv', sep=',', index=False) 
display(df)

,Modelo,Configuracion,Precision,desvPrecision,errorMetrico
65,GradientBoostingClassifier,"[5, 5, 5, 5, 5]",0.860644391408,0.0171459146416,0.173409520148
71,VotingClassifier,"[5, 5, 5, 5, 5]",0.846824639164,0.0189944969668,0.204555890272
47,ExtraTreesClassifier,"[5, 5, 5, 5, 5]",0.842061597909,0.0165258632528,0.209606652995
53,AdaBoostClassifier,"[5, 5, 5, 5, 5]",0.836588816911,0.021882318952,0.234860466608
59,RandomForestClassifier,"[5, 5, 5, 5, 5]",0.836121718377,0.022600658083,0.226442528737
63,GradientBoostingClassifier,"[3, 3, 3, 3, 3]",0.822946200347,0.0189784485903,0.167309588927
62,GradientBoostingClassifier,"[2, 2, 2, 2, 2]",0.819985089887,0.0249329851551,0.141567301909
60,GradientBoostingClassifier,"[0, 0, 0, 0, 0]",0.816516167665,0.0182266763725,0.1582336853
29,KNeighborsClassifier,"[5, 5, 5, 5, 5]",0.810627912263,0.0178548783715,0.227284322524
69,VotingClassifier,"[3, 3, 3, 3, 3]",0.800936355155,0.0197620375111,0.170063656235


In [40]:
df = pd.DataFrame(salida).sort_values(['Accuracy'],ascending=False)
#df.to_csv('resultados_simetricos_sinFiltro.csv', sep=',', index=False) 
display(df[['Modelo', 'Configuracion', 'Accuracy', 'stdAccuracy', 'errorMetrico']])

,Modelo,Configuracion,Accuracy,stdAccuracy,errorMetrico
137,GradientBoostingClassifier,"[6, 6, 6, 6, 6]",0.860644,0.017146,0.173410
143,VotingClassifier,"[6, 6, 6, 6, 6]",0.846825,0.018994,0.204556
119,ExtraTreesClassifier,"[6, 6, 6, 6, 6]",0.842062,0.016526,0.209607
125,AdaBoostClassifier,"[6, 6, 6, 6, 6]",0.836589,0.021882,0.234860
131,RandomForestClassifier,"[6, 6, 6, 6, 6]",0.836122,0.022601,0.226443
135,GradientBoostingClassifier,"[4, 4, 4, 4, 4]",0.822946,0.018978,0.167310
134,GradientBoostingClassifier,"[3, 3, 3, 3, 3]",0.819985,0.024933,0.141567
132,GradientBoostingClassifier,"[1, 1, 1, 1, 1]",0.816516,0.018227,0.158234
101,KNeighborsClassifier,"[6, 6, 6, 6, 6]",0.810628,0.017855,0.227284
141,VotingClassifier,"[4, 4, 4, 4, 4]",0.800936,0.019762,0.170064


In [41]:
#topDf = df.drop_duplicates(subset=['Modelo'])
#display(topDf)
#pd.DataFrame(salida).sort_values(['Accuracy'], ascending=False)
topDf=df.drop_duplicates(subset=['Modelo'])
display(topDf[['Modelo', 'Configuracion', 'Accuracy', 'stdAccuracy', 'errorMetrico']])

,Modelo,Configuracion,Accuracy,stdAccuracy,errorMetrico
137,GradientBoostingClassifier,"[6, 6, 6, 6, 6]",0.860644,0.017146,0.173410
143,VotingClassifier,"[6, 6, 6, 6, 6]",0.846825,0.018994,0.204556
119,ExtraTreesClassifier,"[6, 6, 6, 6, 6]",0.842062,0.016526,0.209607
125,AdaBoostClassifier,"[6, 6, 6, 6, 6]",0.836589,0.021882,0.234860
131,RandomForestClassifier,"[6, 6, 6, 6, 6]",0.836122,0.022601,0.226443
101,KNeighborsClassifier,"[6, 6, 6, 6, 6]",0.810628,0.017855,0.227284
83,SVC,"[6, 6, 6, 6, 6]",0.791816,0.020481,0.250013
107,DecisionTreeClassifier,"[6, 6, 6, 6, 6]",0.781560,0.015619,0.244962
89,GaussianNB,"[6, 6, 6, 6, 6]",0.777047,0.022822,0.309780
77,LinearDiscriminantAnalysis,"[6, 6, 6, 6, 6]",0.666508,0.020948,0.426789


In [61]:
#"""
import plotly
import plotly.graph_objs as go
from plotly.offline.offline import _plot_html
plotly.offline.init_notebook_mode(connected=True)
#"""
#salida = list(topDf.T.to_dict().values())
#names = list(salida[0].keys())
#salida
#
#plot_models(topDf)
#plot_models(df)
#
def plot_models(dataframe):
    names = list(dataframe.columns)
    #names = [i.split('Classifier')[0] for i in dataframe.columns]
    df = dataframe.sort_values(['Accuracy'],ascending=False).drop_duplicates(subset=['Modelo'])
    df['Modelo'] = df['Modelo'].str.replace('Classifier','')
    df['Modelo'] = df['Modelo'].str.replace('LinearDiscriminantAnalysis','LDA')
    salida = list(df.T.to_dict().values())
    results = sorted(salida, key=lambda k: k['Accuracy'], reverse=True)
    data = []
    N = len(results)
    c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 270, N)]
    
    for i, d in enumerate(results):
        trace = go.Box(
            y=d['values'],
            name=d['Modelo'],
            marker=dict(
                color=c[i],
            ),
            boxmean='sd'
        )
        data.append(trace)
    
    text_scatter = go.Scatter(
            x=[d['Modelo'] for d in results],
            y=[d['Accuracy'] for d in results],
            name='score',
            mode='markers',
            text=['Explanation' for _ in results]
    )
    data.append(text_scatter)

    layout = go.Layout(
        #showlegend=False,
        title='Comparison of Clasification Models',
        annotations=[
            dict(
                x=results[0]['Modelo'],
                y=results[0]['Accuracy'],
                xref='x',
                yref='y',
                #text='Best model',
                showarrow=True,
                arrowhead=7,
                ax=0,
                ay=-40
            ),
            dict(
                x=results[-1]['Modelo'],
                y=results[-1]['Accuracy'],
                xref='x',
                yref='y',
                #text='Worst model',
                showarrow=True,
                arrowhead=7,
                ax=0,
                ay=-40
            )
        ]
    )
    fig = go.Figure(data=data, layout=layout)
    
    plotly.offline.iplot(fig, image='svg')

    
plot_models(df)